Python notebook made with Google Collab. Thanks to the article written at this link : https://towardsdatascience.com/deep-learning-for-self-driving-cars-7f198ef4cfa2

### Clonning the project to gather the data

In [1]:
!git clone https://github.com/aurelien-m/HorizonNet.git

Cloning into 'HorizonNet'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 955 (delta 0), reused 0 (delta 0), pack-reused 952
Receiving objects: 100% (955/955), 569.86 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (537/537), done.


### Importing all the librairies we need

In [0]:
import cv2
from google.colab.patches import cv2_imshow

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os

### Down scalling the images and getting the steering angle

In [0]:
!mkdir /content/HorizonNet/data_resized/

In [0]:
downSizeScale = 4

images = [x for x in os.listdir("/content/HorizonNet/data/") if x.endswith(".png")]
for image in images:
  originalImage = cv2.imread('/content/HorizonNet/data/' + str(image), cv2.IMREAD_COLOR)

  newWidth, newHeight = originalImage.shape[1] / downSizeScale, originalImage.shape[0] / downSizeScale
  newImage = cv2.resize(originalImage, (int(newWidth), int(newHeight)))

  cv2.imwrite('/content/HorizonNet/data_resized/' + str(image), newImage)

In [5]:
my_data = pd.read_csv('/content/HorizonNet/data/inputs.csv')
all_data = []

for data_car in my_data[['imagefile', 'x']].iterrows():
  wheel_angle = ((data_car[1]['x'] * 360) / 65535)
  all_data.append((data_car[1]['imagefile'], wheel_angle))

print(all_data)

[('0_image.png', -23.296864271000228), ('1_image.png', 16.30395971618219), ('2_image.png', 59.69512474250401), ('3_image.png', 69.84115358205538), ('4_image.png', 94.74753948271916), ('5_image.png', 75.2300297550927), ('6_image.png', -22.016937514305333), ('7_image.png', 54.43259327077134), ('8_image.png', -28.032043945983062), ('9_image.png', -68.6161593041886), ('10_image.png', 50.840009155413135), ('11_image.png', 64.83131151293202), ('12_image.png', 85.88693064774547), ('13_image.png', 40.44129091325246), ('14_image.png', -24.708628976882583), ('15_image.png', 35.81597619592584), ('16_image.png', 87.81506065461204), ('17_image.png', 85.62874799725337), ('18_image.png', -29.828336003662166), ('19_image.png', -92.56122682536049), ('20_image.png', -94.09384298466469), ('21_image.png', -54.921492332341494), ('22_image.png', -50.31265735866331), ('23_image.png', -60.167544060425726), ('24_image.png', -58.88761730373083), ('25_image.png', 97.05470359349965), ('26_image.png', 132.23346303

### Setting the training and validation datasets

In [0]:
train_len = int(0.8 * len(all_data))
valid_len = len(all_data) - train_len

train_samples, validation_samples = data.random_split(all_data, lengths=[train_len, valid_len])

### Defining the augmentation function
Takes an image with the angle of the steering wheel then crops it and flip it horizontally.

In [0]:
def augment(image_name, angle):
  name = '/content/HorizonNet/data_resized/' + image_name
  current_image = cv2.imread(name)
  current_image = current_image[65:-25, :, :]

  if np.random.rand() < 0.5:
    current_image = cv2.flip(current_image, 1)
    angle = angle * -1.0
  
  return current_image, angle

### Defining the Dataset class

In [0]:
class Dataset(data.Dataset):

  def __init__(self, samples, transform=None):
    self.samples = samples
    self.transform = transform

  def __getitem__(self, index):
    batch_sample = self.samples[index]
    augmented_image, augmented_angle = augment(batch_sample[0], batch_sample[1])
    augmented_image = self.transform(augmented_image)

    #print('augmented image: ', augmented_image.shape) #--> (85, 242, 3)

    return (augmented_image, augmented_angle)

  def __len__(self):
    return len(self.samples)

In [0]:
transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 4}

training_set = Dataset(train_samples, transformations)
training_generator = DataLoader(training_set, **params)

validation_set = Dataset(validation_samples, transformations)
validation_generator = DataLoader(validation_set, **params)

### Defining the network

In [0]:
class NetworkLight(nn.Module):

  def __init__(self):
    super(NetworkLight, self).__init__()
    self.conv_layers = nn.Sequential(
        nn.Conv2d(3, 24, 3, stride=2),
        nn.ELU(),
        nn.Conv2d(24, 48, 3, stride=2),
        nn.MaxPool2d(4, stride=4),
        nn.Dropout(p=0.25)
    )
    self.linear_layers = nn.Sequential(
        nn.Linear(in_features=3360, out_features=50), # 3360 --> figuring out where that came out from ? :o
        nn.ELU(),
        nn.Linear(in_features=50, out_features=10),
        nn.Linear(in_features=10, out_features=1)
    )

  def forward(self, input):
    input = input.view(input.size(0), 3, 85, 242) # input.size() --> torch.Size([32, 85, 242, 3])
    output = self.conv_layers(input)
    output = output.view(output.size(0), -1) # output.size() --> torch.Size([32, 48, 5, 14])
    output = self.linear_layers(output)
    return output

### Setting up the model

In [0]:
model = NetworkLight()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()

In [105]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('device is: ', device)

def toDevice(datas, device):
  imgs, angles = datas
  #print('imgs size: ', imgs.size())
  #print('angles size: ', angles.size())
  return imgs.float().to(device), angles.float().to(device)

device is:  cuda


### Training

In [106]:
max_epochs = 22
for epoch in range(max_epochs):
  model.to(device)

  # Training
  train_loss = 0
  model.train()
  for local_batch, (images, angles) in enumerate(training_generator):
    #print('image size:', image.size()) #--> torch.Size([32, 85, 242, 3])
    #print('image size:', angle.size()) #--> torch.Size([32])

    # Transfer to GPU
    datas = toDevice((images, angles), device)

    # Model computations
    optimizer.zero_grad()

    imgs, angle = datas
    #print("training image: ", imgs.shape)
    outputs = model(imgs)
    loss = criterion(outputs, angle.unsqueeze(1))
    loss.backward()
    optimizer.step()

    train_loss += loss.data.item()

    if local_batch % 100 == 0:
      print('Loss: %.3f ' % (train_loss/(local_batch+1)))

Loss: 4355.042 
Loss: 7121.093 
Loss: 5509.562 
Loss: 5053.421 
Loss: 3369.875 
Loss: 5006.351 
Loss: 4903.132 
Loss: 5043.459 
Loss: 4999.727 
Loss: 5406.239 
Loss: 5462.854 
Loss: 3193.299 
Loss: 4903.310 
Loss: 4827.616 
Loss: 3823.911 
Loss: 5661.630 
Loss: 5001.299 
Loss: 5108.333 
Loss: 4887.521 
Loss: 5156.684 
Loss: 3469.420 
Loss: 5624.591 


### Validation

In [119]:
model.eval()
valid_loss = 0
with torch.set_grad_enabled(False):
  for local_batch, (images, angles) in enumerate(validation_generator):
    # Transfer to GPU
    datas = toDevice((images, angles), device)
    imgs, angles = datas

    # Model computations
    optimizer.zero_grad()

    #print("Validation image: ", imgs.shape)
    outputs = model(imgs)
    loss = criterion(outputs, angles.unsqueeze(1))
    
    valid_loss += loss.data.item()

    if local_batch % 100 == 0:
      print('Valid Loss: %.3f ' % (valid_loss/(local_batch+1)))

Valid Loss: 4486.925 


In [121]:
state = {
  'model': model.module if device == 'cuda' else model,
}

torch.save(state, '/content/HorizonNet/horizon_net.h5')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type NetworkLight. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ELU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/